In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, time, datetime, re

In [2]:
files = glob.glob("../data/springfield_springfield_all/"+"*.csv")
shows = pd.concat([pd.read_csv(file, lineterminator='\n') for file in files])

In [3]:
shows.shape

(117375, 3)

In [4]:
shows.head()

,text,episode_name,show_name
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,'Allo 'Allo! (1982) s03e01 Episode Script
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,'Allo 'Allo! (1982) s03e02 Episode Script
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,'Allo 'Allo! (1982) s03e03 Episode Script
3,It is possible that\r the more curious among y...,The Flight of Fancy,'Allo 'Allo! (1982) s03e04 Episode Script
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,'Allo 'Allo! (1982) s03e05 Episode Script


In [5]:
shows.tail()

,text,episode_name,show_name
42093,: Out of the night When the full moon ls brigh...,The Flaming Arrow,Zorro (1957) s02e11 Episode Script
42094,: Out of the night When the full moon ls brigh...,Zorro Fights a Duel,Zorro (1957) s02e12 Episode Script
42095,: Out of the night When the full moon ls brigh...,Amnesty for Zorro,Zorro (1957) s02e13 Episode Script
42096,: Out of the night When the full moon ls brigh...,The Runaways,Zorro (1957) s02e14 Episode Script
42097,: Out of the night When the full moon ls brigh...,The Iron Box,Zorro (1957) s02e15 Episode Script


In [6]:
shows.describe()

,text,episode_name,show_name
count,117225,117371,117374
unique,116735,88890,117315
top,"1\r Happy New Year 2015 - New Year, New Color ...",Episode 1,The A-Team (1983) Episode Scripts
freq,4,893,15


In [7]:
shows.isnull().sum()

text            150
episode_name      4
show_name         1
dtype: int64

In [8]:
shows[shows["text"].isnull()]["show_name"].value_counts()

The A-Team (1983) Episode Scripts                      15
Shaun the Sheep (2007) Episode Scripts                  9
TV Show Episode Scripts                                 3
COPS (1989) s26e16 Episode Script                       1
COPS (1989) s26e06 Episode Script                       1
                                                       ..
Lilyhammer (2011) Episode Scripts                       1
1000 Ways to Die s05e03 Episode Script                  1
COPS (1989) s27e11 Episode Script                       1
Bella and the Bulldogs (2015) s01e07 Episode Script     1
Underbelly s06e07 Episode Script                        1
Name: show_name, Length: 125, dtype: int64

#### Cleaning Show Name Column

In [9]:
shows["show_name"] = shows["show_name"].str.replace("Episode Script", "")

In [10]:
shows["show_name_split"] = shows["show_name"].str.split("(")

In [11]:
shows["show_name"] = shows["show_name_split"].str.get(0)

In [12]:
shows["show_name"] = shows["show_name"].str.replace("\'", "")

In [13]:
shows.head()

,text,episode_name,show_name,show_name_split
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,Allo Allo!,"['Allo 'Allo! , 1982) s03e01 ]"
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,Allo Allo!,"['Allo 'Allo! , 1982) s03e02 ]"
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,Allo Allo!,"['Allo 'Allo! , 1982) s03e03 ]"
3,It is possible that\r the more curious among y...,The Flight of Fancy,Allo Allo!,"['Allo 'Allo! , 1982) s03e04 ]"
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,Allo Allo!,"['Allo 'Allo! , 1982) s03e05 ]"


In [14]:
shows["year_season_episode"] = shows["show_name_split"].str.get(1)

In [15]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,Allo Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,Allo Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,Allo Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03
3,It is possible that\r the more curious among y...,The Flight of Fancy,Allo Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,Allo Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05


In [16]:
shows["year_season_episode_split"] = shows["year_season_episode"].str.split(" ")

In [17]:
shows["year"] = shows["year_season_episode_split"].str.get(0).str.replace(")", "")

In [18]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode,year_season_episode_split,year
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,Allo Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01,"[1982), s03e01, ]",1982
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,Allo Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02,"[1982), s03e02, ]",1982
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,Allo Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03,"[1982), s03e03, ]",1982
3,It is possible that\r the more curious among y...,The Flight of Fancy,Allo Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04,"[1982), s03e04, ]",1982
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,Allo Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05,"[1982), s03e05, ]",1982


In [19]:
shows["season_episode_split"] = shows["year_season_episode_split"].str.get(1).str.split("e")

In [20]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode,year_season_episode_split,year,season_episode_split
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,Allo Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01,"[1982), s03e01, ]",1982,"[s03, 01]"
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,Allo Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02,"[1982), s03e02, ]",1982,"[s03, 02]"
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,Allo Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03,"[1982), s03e03, ]",1982,"[s03, 03]"
3,It is possible that\r the more curious among y...,The Flight of Fancy,Allo Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04,"[1982), s03e04, ]",1982,"[s03, 04]"
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,Allo Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05,"[1982), s03e05, ]",1982,"[s03, 05]"


In [21]:
shows["season"] = shows["season_episode_split"].str.get(0).str.replace("s", "")

In [22]:
shows["episode_number"] = shows["season_episode_split"].str.get(1)

In [23]:
shows.head()

,text,episode_name,show_name,show_name_split,year_season_episode,year_season_episode_split,year,season_episode_split,season,episode_number
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,Allo Allo!,"['Allo 'Allo! , 1982) s03e01 ]",1982) s03e01,"[1982), s03e01, ]",1982,"[s03, 01]",03,01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,Allo Allo!,"['Allo 'Allo! , 1982) s03e02 ]",1982) s03e02,"[1982), s03e02, ]",1982,"[s03, 02]",03,02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,Allo Allo!,"['Allo 'Allo! , 1982) s03e03 ]",1982) s03e03,"[1982), s03e03, ]",1982,"[s03, 03]",03,03
3,It is possible that\r the more curious among y...,The Flight of Fancy,Allo Allo!,"['Allo 'Allo! , 1982) s03e04 ]",1982) s03e04,"[1982), s03e04, ]",1982,"[s03, 04]",03,04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,Allo Allo!,"['Allo 'Allo! , 1982) s03e05 ]",1982) s03e05,"[1982), s03e05, ]",1982,"[s03, 05]",03,05


In [24]:
shows = shows.drop(columns=["show_name_split", "year_season_episode", "year_season_episode_split", "season_episode_split"])

In [25]:
shows.head()

,text,episode_name,show_name,year,season,episode_number
0,Can it be the breeze that fills the trees\r W...,The Nicked Knockwurst,Allo Allo!,1982,03,01
1,Madame Peyet is in the family way.\r She is p...,Gruber Does Some Mincing,Allo Allo!,1982,03,02
2,Ooar!\r Ooh!\r Are you by any chance curious...,The Sausage in the Wardrobe,Allo Allo!,1982,03,03
3,It is possible that\r the more curious among y...,The Flight of Fancy,Allo Allo!,1982,03,04
4,Oh. Do you by any chance wish to know\r how I...,Pretty Maids All in a Row,Allo Allo!,1982,03,05


In [26]:
shows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117375 entries, 0 to 42097
Data columns (total 6 columns):
text              117225 non-null object
episode_name      117371 non-null object
show_name         117374 non-null object
year              72418 non-null object
season            72418 non-null object
episode_number    71372 non-null object
dtypes: object(6)
memory usage: 6.3+ MB


#### Cleaning the Show Name Column 

There is one row that has no show name, and when inspected, is a completely null row. Will drop

In [35]:
shows[shows["show_name"].isnull()].shape

(1, 6)

In [36]:
shows = shows.drop(shows[shows["show_name"].isnull()].index)

In [37]:
shows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117373 entries, 0 to 42097
Data columns (total 6 columns):
text              117224 non-null object
episode_name      117370 non-null object
show_name         117373 non-null object
year              72417 non-null object
season            72417 non-null object
episode_number    71371 non-null object
dtypes: object(6)
memory usage: 6.3+ MB


In [44]:
shows["show_name"].value_counts()

Doctor Who                       702
Naruto: Shippuden                500
Law & Order                      456
Teenage Mutant Ninja Turtles     315
Last of the Summer Wine          287
                                ... 
Gossip Girl s04e10                 1
The Chicago Code s01e13            1
Everwood s01e14                    1
Party of Five s06e07               1
Psych s04e08                       1
Name: show_name, Length: 48869, dtype: int64

The NaN values in the year, season, and episode_number are due to a different show_name format. Will split the dataframe to clean these show names, fill in the missing values with the ones in the show names and then recombine with the already cleaned data 

In [55]:
#pulling out show_names with a number
mask = shows["show_name"].str.contains("\d")